In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import os
from scrape_utils import get_tm_team_links, get_tm_team_squad

# Scrape transfermarkt data for the big-5 leagues

Get the links for each football team in the big-5 leagues for the 2019 season

In [2]:
league_urls = ['https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1/plus/?saison_id=2020',
               'https://www.transfermarkt.com/laliga/startseite/wettbewerb/ES1/plus/?saison_id=2020',
               'https://www.transfermarkt.com/bundesliga/startseite/wettbewerb/L1/plus/?saison_id=2020',
               'https://www.transfermarkt.com/serie-a/startseite/wettbewerb/IT1/plus/?saison_id=2020',
               'https://www.transfermarkt.com/ligue-1/startseite/wettbewerb/FR1/plus/?saison_id=2020']

team_links = []
for url in league_urls:
    links = get_tm_team_links(url)
    team_links.extend(links)
    time.sleep(np.random.uniform(low=2, high=5))
    
print('Number of teams:', len(team_links))

Number of teams: 98


Replace the compact (startseite) with the detailed version (kader). And add on plus/1 to remove some of the extra tables

In [3]:
team_links = [s.replace('startseite', 'kader') for s in team_links] 
# not needed as looking at current season anyway
# team_links.extend([f'{s[:-4]}2020' for s in team_links])  # add 2020 pages for latest market values
team_links = [f'{l}/plus/1' for l in team_links]

Get a dataframe for each team with the player data (big-5 leagues)

In [4]:
team_dfs = []
for url in team_links:
    year = url[-11:-7]
    df = get_tm_team_squad(url)
    df['year'] = year
    fname = os.path.join('data', 'transfermarkt', url.split('/')[3] + '.parquet')
    df.to_parquet(fname)
    time.sleep(np.random.uniform(low=2, high=5))